In [1]:
#import everything and set dpi here

import os
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 100 #set dpi of figs for entire notebook

In [10]:
directory = r"C:/Users/njana/Desktop/4simbaResults/Day1/"
infofile = r"C:/Users/njana/Desktop/4simbaResults/Day1/day1.csv"
infotable = pd.read_csv(infofile)
infotable
# "Hardware-introNbutton-Trial    12-Arena 1.txt"

,name,type,Day,Trial,stimdur,Total,startf,endf,trunc end,Mouse,ethofile
0,D1T1,standard,1,1,30,39618,5469,39618,0,3,Hardware-Mar2223-Trial 1-Arena 1
1,D1T2,standard,1,2,30,39613,4490,39613,0,6,Hardware-Mar2223-Trial 2-Arena 1
2,D1T3,standard,1,3,30,39621,5499,39621,0,2,Hardware-Mar2223-Trial 3-Arena 1
3,D1T4,standard,1,4,30,39614,5279,39614,0,5,Hardware-Mar2223-Trial 4-Arena 1
4,D1T5,standard,1,5,30,39609,5409,39609,0,1,Hardware-Mar2223-Trial 5-Arena 1
5,D1T6,standard,1,6,30,39622,5348,39622,0,4,Hardware-Mar2223-Trial 6-Arena 1


In [11]:
''' folder plotter WITH averages: not for day 3 random stuff''' 

for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_contact",	"Social_contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_move",	"Rest_move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37) # so stupid that this works but it does

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
        
    Mouse = "A0"+str(infotable.Mouse[i])
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
    avgtime = np.arange(0,stimdur*3*fps)/fps

    
    colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
    sns.set_style("white")

    #plt.subplots(9,2,gridspec_kw={'width_ratios': [time.shape[0],stimdur*fps*3]})
    plt.subplots(9,2,gridspec_kw={'width_ratios': [15,1]})
    #plt.subplots_adjust(hspace=0)
    plt.subplots_adjust(wspace=0)


    plt.subplot(9,2,1)
    plt.plot(time,stim, color = 'blue')
    plt.ylabel("Stimulus", color = 'blue')
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    
    plt.subplot(9,2,2)
    plt.plot(avgtime,stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)], color = 'blue')
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    
    counter = 0
    plotnum = 3

    for k in range(8):
        
        plt.subplot(9,2,plotnum)
        plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[k], alpha = 0.3)
        counter += 1
        
        plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[k], alpha = 1)
        plt.ylabel(behavdata.columns[counter], color = colors[k])
        if plotnum < 16:
            plt.gca().set_xticks([]) 
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)
        plotnum += 1
        
        plt.subplot(9,2,plotnum)
        
        # calculate the average data
        behav = behavdata[behavdata.columns[counter]].to_numpy()
        window_behav = []
        for ele in stimstartidx:
            window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
        window_behav = np.array(window_behav)
        avg_behav = np.average(window_behav, axis = 0)
    
        # plot the average data
        plt.plot(avgtime,avg_behav, color = colors[k], alpha = 1)
        plt.ylim(0,1)
        if plotnum < 16:
            plt.gca().set_xticks([])
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)        
            
        counter += 1   
        plotnum += 1
    
    plt.gcf().set_size_inches(16,8)
    plt.gcf().suptitle("Resident "+Mouse+": Day "+day+" Trial "+trial,x = 0.225, y = 0.92, fontsize = 16, fontweight = "bold")

    plt.savefig(directory+Mouse+"D"+day+"Tr"+trial+".svg", dpi = 500)
    plt.savefig(directory+Mouse+"D"+day+"Tr"+trial+".png", dpi = 500)
    
    plt.clf()

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

<Figure size 1600x800 with 0 Axes>

### OLD folder plotter without averages

In [ ]:
''' OLD folder plotter without averages ''' 

for i in range(infotable.shape[0]):
    #get the data
    simbarawdata = pd.read_csv(directory+infotable.name[i]+".csv", header = 0)

    behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                            "Probability_Mounting",	"Mounting", 
                            "Probability_Following",	"Following",	
                            "Probability_Dominate",	"Dominate",	
                            "Probability_Social_contact",	"Social_contact",	
                            "Probability_Asocial",	"Asocial",	
                            "Probability_Passive",	"Passive",	
                            "Probability_Rest_move",	"Rest_move"]]
    
    ethoexportpath = directory+infotable.ethofile[i]+".txt"
    ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37) # so stupid that this works but it does

    input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
    input1high = input1.loc[input1["Value"] == 1]
    input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

    input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
    input2high = input2.loc[input2["Value"] == 1]
    input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
    
    stimtimes = input2high["Recording time"].to_numpy()
    stimtimes = np.asarray(stimtimes, dtype=float)

    truncated = infotable.startf[i]
    fps = 30
    timetrunc = truncated/fps
    stimtimes = stimtimes - timetrunc
    
    stimstartidx = np.around(stimtimes*fps).astype("int")

    stimdur = infotable.stimdur[i] #in seconds
    stimidxdur = stimdur*fps
    stimendidx = stimstartidx + stimidxdur

    stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

    for j in range(stimstartidx.shape[0]):
        stim[stimstartidx[j]:stimendidx[j]] = 1
        
    Mouse = "A0"+str(infotable.Mouse[i])
    day = str(infotable.Day[i])
    trial = str(infotable.Trial[i])
    time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps

    counter = 0
    colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
    sns.set_style("white")


    #plt.subplots_adjust(hspace=0)

    plt.subplot(9,1,1)
    plt.plot(time,stim, color = 'blue')
    plt.ylabel("Stimulus", color = 'blue')
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])

    for k in range(8):
        plt.subplot(9,1,k+2)
        plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[k], alpha = 0.3)
        counter += 1
        plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[k], alpha = 1)
        plt.ylabel(behavdata.columns[counter], color = colors[k])
        plt.gca().set_xticks([]) 
        plt.gca().set_yticks([])
        sns.despine(bottom = True, left = True)
        counter += 1   
    
    plt.gcf().set_size_inches(16,8)
    plt.gcf().suptitle("Resident "+Mouse+": Day "+day+" Trial "+trial,x = 0.225, y = 0.92, fontsize = 16, fontweight = "bold")

    plt.savefig(directory+Mouse+"D"+day+"Tr"+trial+".svg", dpi = 500)
    plt.savefig(directory+Mouse+"D"+day+"Tr"+trial+".png", dpi = 500)
    
    plt.clf()

## OLD code to plot single file

In [ ]:
# load simba output
simbaresultpath = r"C:/Users/njana/Desktop/4simbaResults/Day1/D1T1.csv"

simbarawdata = pd.read_csv(simbaresultpath, header = 0)

behavdata = simbarawdata[["Probability_Aggression","Aggression",	
                        "Probability_Mounting",	"Mounting", 
                        "Probability_Following",	"Following",	
                        "Probability_Dominate",	"Dominate",	
                        "Probability_Social_contact",	"Social_contact",	
                        "Probability_Asocial",	"Asocial",	
                        "Probability_Passive",	"Passive",	
                        "Probability_Rest_move",	"Rest_move"]]

In [ ]:
# load corresponding ethovision export

ethoexportpath = r"C:/Users/njana/Desktop/4simbaResults/Day1/Hardware-Mar2223-Trial     1-Arena 1.txt"
ethoexport = pd.read_csv(ethoexportpath, sep = ';', header = 37) # so stupid that this works but it does

input1 = ethoexport.loc[ethoexport["Name"] == "Is input 1 High"]
input1high = input1.loc[input1["Value"] == 1]
input1 = input1.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
input1high = input1high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

input2 = ethoexport.loc[ethoexport["Name"] == "Is input 2 High"]
input2high = input2.loc[input2["Value"] == 1]
input2 = input2.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])
input2high = input2high.drop(columns = ['Trial time', 'Device', 'Command/Signal', 'Unnamed: 6'])

In [ ]:
stimtimes = input2high["Recording time"].to_numpy()
stimtimes = np.asarray(stimtimes, dtype=float)

truncated = 5469 # for d1t1

fps = 30
timetrunc = truncated/fps
stimtimes = stimtimes - timetrunc

In [ ]:
stimstartidx = np.around(stimtimes*fps).astype("int")

stimdur = 30 #in seconds
stimidxdur = stimdur*fps
stimendidx = stimstartidx + stimidxdur

stim = np.zeros(behavdata[behavdata.columns[0]].shape[0])

for i in range(stimstartidx.shape[0]):
    stim[stimstartidx[i]:stimendidx[i]] = 1

In [ ]:
""" Simple plot without averages """

Mouse = "A03"
day = "1"

fps = 30
time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps

counter = 0
colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
sns.set_style("white")


#plt.subplots_adjust(hspace=0)

plt.subplot(9,1,1)
plt.plot(time,stim, color = 'blue')
plt.ylabel("Stimulus", color = 'blue')
plt.gca().set_xticks([]) 
plt.gca().set_yticks([])

for i in range(8):
    plt.subplot(9,1,i+2)
    plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[i], alpha = 0.3)
    counter += 1
    plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[i], alpha = 1)
    plt.ylabel(behavdata.columns[counter], color = colors[i])
    plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    sns.despine(bottom = True, left = True)
    counter += 1   
    
plt.gcf().set_size_inches(16,8)
plt.gcf().suptitle("Resident "+Mouse+": Day "+day,x = 0.225, y = 0.92, fontsize = 16, fontweight = "bold")

#plt.savefig(Mouse+"D"+day+".svg", dpi = 500)
#plt.savefig(Mouse+"D"+day+".png", dpi = 500)

In [ ]:
''' Plot with averages on the right '''

Mouse = "A03"
day = "1"

fps = 30
time = np.arange(0,behavdata[behavdata.columns[0]].shape[0])/fps
avgtime = np.arange(0,stimdur*3*fps)/fps


colors = ["red","brown","orange","purple","green","turquoise","black","sandybrown"]
sns.set_style("white")

plt.subplots(9,2,gridspec_kw={'width_ratios': [time.shape[0],stimdur*fps*3]})
plt.subplots_adjust(hspace=0)
plt.subplots_adjust(wspace=0)

plt.subplot(9,2,1)
plt.plot(time,stim, color = 'blue')
plt.ylabel("Stimulus", color = 'blue')
plt.gca().set_xticks([]) 
plt.gca().set_yticks([])

plt.subplot(9,2,2)
plt.plot(avgtime,stim[stimstartidx[0]-(stimdur*fps):stimstartidx[0]+(stimdur*fps*2)], color = 'blue')
plt.gca().set_xticks([]) 
plt.gca().set_yticks([])


counter = 0
plotnum = 3
for i in range(8):
    # plot the entire range on the left
    plt.subplot(9,2, plotnum)
    
    # plot the probability trace
    plt.plot(time,behavdata[behavdata.columns[counter]],  color = colors[i], alpha = 0.3)
    counter += 1
    
    # plot the behaviour occurences
    plt.plot(time,behavdata[behavdata.columns[counter]], color = colors[i], alpha = 1)
    plt.ylabel(behavdata.columns[counter], color = colors[i])
    if plotnum < 16:
        plt.gca().set_xticks([]) 
    plt.gca().set_yticks([])
    sns.despine(bottom = True, left = True)
    
    plotnum += 1
    
    # plot the averages on the right    
    plt.subplot(9,2,plotnum)
    
    # calculate the average data
    behav = behavdata[behavdata.columns[counter]].to_numpy()
    window_behav = []
    for ele in stimstartidx:
        window_behav += [behav[ele-(stimdur*fps):ele+(stimdur*fps*2)]]
    
    window_behav = np.array(window_behav)
    avg_behav = np.average(window_behav, axis = 0)
    
    # plot the average data
    plt.plot(avgtime,avg_behav, color = colors[i], alpha = 1)
    plt.ylim(0,1)
    if plotnum < 16:
        plt.gca().set_xticks([])
    plt.gca().set_yticks([])
    sns.despine(bottom = True, left = True)
    counter += 1
    plotnum += 1
    
plt.gcf().set_size_inches(16,8)
plt.gcf().suptitle("Resident "+Mouse+": Day "+day,x = 0.225, y = 0.92, fontsize = 16, fontweight = "bold")

#plt.savefig(Mouse+"D"+day+".svg", dpi = 500)
#plt.savefig(Mouse+"D"+day+".png", dpi = 500)